In [1]:
import sys
sys.path.append("../code")
from dataset import build_dataset, build_dataloader
from config import DataArguments, TrainerArguments, ModelArguments

In [2]:
data_args = DataArguments
training_args = TrainerArguments
model_args = ModelArguments

data_args.data_dir = "../data/"
data_args.seed = training_args.seed
data_args.max_seq_len = model_args.max_seq_len

data_args.init_pct = 0.01

train_dataset, model_args.vocab_size, model_args.num_labels = build_dataset(data_args, "train")
train_dataloader = build_dataloader(train_dataset, data_args)

[11/21/2021 14:43:06] INFO - dataset: Initialize Train Dataset.
[11/21/2021 14:43:06] INFO - dataset: Remove abstract.
[11/21/2021 14:43:06] INFO - dataset: Remove task_id
[11/21/2021 14:43:06] INFO - dataset: Use 0.01% of the total dataset.
[11/21/2021 14:43:07] INFO - dataset: Total 473 of papers will be used.
[11/21/2021 14:43:07] INFO - dataset: Train dataset was successfully initialized.
[11/21/2021 14:43:10] INFO - preprocess: Successfully loaded Spacy Tokenizer, en_core_web_trf
[11/21/2021 14:43:10] INFO - dataset: Successfully loaded mapper file ..\assets\area2idx.json
[11/21/2021 14:43:14] WARNING - datasets.fingerprint: Parameter 'function'=<function build_dataset.<locals>.batch_encode at 0x000001F2BA064048> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this tra

In [3]:
from trainer import NaiveTrainer

training_args.do_train = True

trainer = NaiveTrainer(
    training_args,
    model_args,
    training_dataset=train_dataloader if training_args.do_train else None,
    validation_dataset=valid_dataloader if training_args.do_valid else None,
    test_dataset=test_dataloader if training_args.do_test else None,
)

[11/21/2021 14:43:45] INFO - trainer: Bert was selected. Start Transformer setup.
[11/21/2021 14:43:45] INFO - trainer: Successfully setup transformer settings.


In [4]:
trainer.train()

RuntimeError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 6.00 GiB total capacity; 4.51 GiB already allocated; 0 bytes free; 4.58 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF